# Unstructured 文件解析與多模態資料處理

---

**本章學完你將能學會什麼：**

- 理解什麼是 *Unstructured* 套件，以及它在文件解析與資料前處理中的應用場景。  
- 學會如何在 Google Colab（Linux 環境）中安裝與設定 *Unstructured* 套件，正確抽取 PDF、表格與影像資料。  
- 掌握多種文件解析策略（`fast`、`hi_res`、`ocr_only`）的差異與最佳使用情境。  
- 學會使用不同的分段策略（chunking strategy），例如 `by_title`，將長篇文件切割成具語意邏輯的文字區塊。  
- 理解如何從影像與表格中萃取結構化資訊，並運用 OCR（光學字元辨識）技術。  
- 學會將抽取後的內容轉換成可搜尋的向量資料庫（VectorStore），為後續 RAG（Retrieval-Augmented Generation）應用奠定基礎。  
- 熟悉如何整合 LangChain、OpenAI API、HuggingFace Embedding 模型，建立從文件 → 向量 → 智慧檢索的完整流程。

---

**📘 最終你將具備的能力：**  
- 能夠撰寫 Python 程式，實現從 PDF 文件中自動抽取文字、影像與表格，並將其轉換為結構化資料。  
- 能理解並應用不同文件解析策略，根據資料特性調整擷取邏輯。  
- 能獨立建立文件資料的向量化流程（FAISS + LangChain），實作文件摘要與圖像文字生成（Image Captioning）。  
- 具備設計多模態 LLM pipeline 的能力，讓模型能根據圖片或文字內容生成描述、摘要或回答問題。

---



Unstructured 算是應用範圍蠻廣泛的Package，支援處理各式各樣的檔案格式，包含了PDF，PPT，HTML等等

無法在Windows系統上成功使用，所以我們將環境換成Google Colab，因為Colab的操作系統是Linux

首先我們要安裝各式各樣的套件

In [ ]:
!apt-get update

---

## ⚙️ 環境設定補充說明

由於 **Unstructured** 這個套件在不同作業系統上相依的元件略有差異，  
在 Windows 上常會因為缺乏底層的 PDF 或 OCR 處理函式庫而導致安裝失敗。  
因此，本章的範例我們統一採用 **Google Colab** 執行環境（Linux 系統），確保相容性。

以下是各套件的用途說明，讓你清楚知道為何要安裝這些依賴：

| 指令 / 套件 | 功能說明 |
|--------------|-----------|
| `poppler-utils` | 提供 PDF 處理工具，例如 `pdftotext`、`pdfimages`，是 PDF 轉換的基礎套件。 |
| `libleptonica-dev` | 圖像處理函式庫，為 OCR 引擎（tesseract）的底層依賴。 |
| `tesseract-ocr`、`libtesseract-dev` | Google 的 OCR 引擎，用於將圖片中的文字轉換為可讀文字。 |
| `python3-pil` | Python 影像處理函式庫（Pillow 的底層依賴）。 |
| `tesseract-ocr-eng`、`tesseract-ocr-script-latn` | OCR 模組，支援英文與拉丁字母辨識。 |
| `unstructured[all-docs]` | 主角套件，支援多種文件格式（PDF、HTML、DOCX、PPTX 等）。 |
| `unstructured-inference` | 內含文件結構辨識模型，用於 `hi_res` 策略。 |
| `pdf2image` | 將 PDF 頁面轉換成影像格式，方便 OCR 或版面分析。 |
| `faiss-cpu` | Facebook AI 相似度搜尋庫，用於向量檢索。 |
| `sentence-transformers` | 提供多種文字向量化模型，例如 BERT、bge-m3。 |
| `langchain`, `langchain-community`, `langchain-core`, `langchain-openai` | 建立 LLM 流程與模組串接的核心工具。 |
| `bitsandbytes`, `accelerate`, `xformers`, `triton`, `transformers` | 深度學習加速與模型推論框架，用於高效運算與模型加速。 |
| `nltk` | 傳統 NLP 工具包，用於文字處理與分詞。 |

> 💡 **提示：**
> 若在安裝過程中出現 `E: Unable to locate package` 或 `ImportError` 錯誤，可嘗試重新執行 `!apt-get update` 後再安裝，  
> 或檢查 Colab 執行環境是否為「Python 3 + GPU / CPU」版本（非 TPU）。

---



In [ ]:
!apt-get install poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
!pip install unstructured[all-docs] unstructured-inference cmake python-dotenv pdf2image python-dateutil faiss-cpu sentence-transformers langchain==0.2.5 langchain-community==0.2.5 langchain-core==0.2.9 langchain-openai==0.1.9 bitsandbytes accelerate xformers triton transformers

In [ ]:
!pip install -U nltk

In [ ]:
import shutil

## 處理PDF

在使用上有點繁瑣，所以我們來一步步地進行測試

- 使用Transformer is all you need這篇論文作為輸入的數據
- 不同的數據處理策略(strategy)
  >- fast: 「基於規則（rule-based）」的策略運用傳統的自然語言處理（NLP）抽取技術，能快速擷取所有文字元素。不建議在以影像為主的檔案類型中使用「fast」策略。
  >- hi_res: 基於模型（model-based）」的策略會辨識文件的版面配置。「hi_res」的優點在於它利用文件的版面結構，獲取關於文件元素的額外資訊。 若您的使用情境對文件元素的正確分類非常敏感，建議使用此策略。
  >- ocr_only: 另一種「基於模型（model-based）」的策略，利用光學字元辨識（OCR）技術，從影像型文件中擷取文字。

一般來說是建議用`hi_res`，因為可以擷取影像資料

In [ ]:
filename = "bertv2.pdf"

In [ ]:
import os
import pathlib

from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

# 參數 max_characters, new_after_n_chars, combine_text_under_n_chars 只有在chunking_strategy不為None時才會生效

elements_none = partition_pdf(
    filename,
    strategy='hi_res',
)

前20個物件

In [ ]:
elements_none[:20]

檢查category attribute

In [ ]:
for element in elements_none[:20]:
    print(element.category)

檢查text attribute

In [ ]:
for element in elements_none[:20]:
    print(element.text)

你可以看得很清楚，沒有chunking strategy訊息散的到處都是，跟侏儸紀公園2中運送暴龍的運輸船上的工作人員一樣。

In [ ]:
import numpy as np

np.unique([element.category for element in elements_none])

## by_title 分段策略（chunking strategy）

`by_title` 分段策略會保留章節邊界，並可選擇同時保留頁面邊界。  
在這裡，「保留」的意思是：同一個區塊（chunk）永遠不會包含來自兩個不同章節的文字。  
當新的章節開始時，現有的區塊會被關閉並開啟新的區塊，即使下一個元素的長度仍可容納於先前的區塊中。

除了上述基本策略的行為之外，`by_title` 策略還具有以下特性：

---

### 🔹 偵測章節標題

當偵測到 **Title** 元素時，該元素會被視為新章節的開始。  
當遇到 **Title** 元素時，前一個區塊會被關閉，並開啟新的區塊，即使該 **Title** 元素的長度可容納於前一個區塊中。

---

### 🔹 遵守頁面邊界

可以使用 `multipage_sections` 參數來選擇是否遵守頁面邊界。  
此參數的預設值為 `True`，表示頁面分隔符（page break）**不會**導致新的區塊開始。  
若將其設為 `False`，則會將位於不同頁面的元素分割為獨立的區塊。

---

### 🔹 合併小型章節

在某些文件中，分段過程可能會將清單項目或其他短段落誤判為 **Title** 元素，  
即使它們實際上並非章節標題。這可能導致產生的區塊比預期的小得多。  

此行為可透過 `combine_text_under_n_chars` 參數進行調整。  
該參數預設值與 `max_characters` 相同，表示會將連續的小章節合併，以最大化區塊的填充程度。  
若將此參數設為 `0`，則會停用章節合併功能。

先來檢查沒有chunking strategy 時數據切割的樣子

In [ ]:
for idx, element in enumerate(elements_none):
    if element.category == "Title":
        print(idx, element)

## chunking_strategy='by_title'

https://docs.unstructured.io/ui/chunking#combine-text-under-n-characters-setting

### max_characters

此參數為單一區塊（chunk）中可包含文字數量的**硬性上限**。  

它確保每個區塊都不會超過此限制，無論句子或段落邊界為何。  

可將其視為區塊大小的「最大界限」。  

👉 **範例：** 若設定 `max_characters=500`，則任何區塊都不會包含超過 500 個字元。

---

### new_after_n_chars

此參數為**柔性分段依據**，在文字長度達到此門檻時，會嘗試建立新的區塊。  

與 `max_characters` 不同的是，它允許函式庫在該範圍內尋找「自然的分隔點」，  
例如句子或段落的結尾。  

當您希望區塊大致維持在某個長度，同時仍能尊重自然語意邊界時，此設定非常有用。  

👉 **範例：** 若設定 `new_after_n_chars=300`，函式庫會嘗試在約 300 個字元後開始新的區塊，  
但若下一個自然分隔點位於第 320 個字元，則可能略為超出此值。

---

### combine_text_under_n_chars

該參數預設值與 `max_characters` 相同，表示會將連續的小章節合併，以最大化區塊的填充程度。  
若將此參數設為 `0`，則會停用章節合併功能。

講解完後實操一次

In [ ]:
elements_by_title = partition_pdf(
    filename,
    strategy='hi_res',
    chunking_strategy='by_title',
)

檢查 category attribute

In [ ]:
for element in elements_by_title[:20]:
  print(element.category)

## 擷取影像和表格

In [ ]:
elements_by_title = partition_pdf(
    filename,
    chunking_strategy='by_title',
    extract_image_block_types=["Image", "Table"],
    strategy='hi_res',
    extract_image_block_output_dir=pathlib.Path(filename).stem
)

你可以看到在這個例子中，影像抽取的結果蠻糟糕的。可能是因為圖像中元素有強烈的邊界，導致影像識別模型無法正確判斷影像。

再試另外一個含有大量照片的PDF檔

In [ ]:
elements_by_title = partition_pdf(
    'menu.pdf',
    chunking_strategy='by_title',
    extract_image_block_types=["Image", "Table"],
    strategy='hi_res',
    extract_image_block_output_dir=pathlib.Path("menu.pdf").stem
)

你可以看到，若是圖像比較像是照片，結果會比較好。但依然會有一切false positive出現。所以你必須要根據數據的類型，決定用哪種抽取方式。

In [ ]:
print(elements_by_title[0].text)

你也許會很好奇 "best quality knowledge and skill" 這段文字哪裡來的。
所以我們退後一步，看看原始數據長啥樣子。

In [ ]:
elements_none = partition_pdf(
    'menu.pdf',
    strategy='hi_res'
)

In [ ]:
for element in elements_none[:20]:
    print(element.category, element.text)

In [ ]:
partition_pdf?

從內容來看，擷取影像的時候也包含了OCR: 會把影像中的文字提取出來，並且在 chunking_strategy中被併入文字訊息的一部分。
此外，你也可以看到在排版上有些奇怪的結果: 像是
- NarrativeText Loaded tortilla-coated crunchy wings seasoned with our signature blend of spices. Choose from:
- Title LOVED BY YOU

就像是直接把兩個左右相鄰的板塊視為上下文的關係。這種情況在雙列排版的內容很常見，所以我們要如何減輕這種情況?

In [ ]:
elements_strategy_ocr_only = partition_pdf(
    'menu.pdf',
    strategy='ocr_only',
    chunking_strategy='by_title'
)

In [ ]:
for element in elements_strategy_ocr_only[:5]:
  print("******")
  print(element.category, element.text)

### 一個便當吃不飽，可以吃兩個

這樣子我們可以看到使用ocr_only這種策略，對於抽取雙列排版的檔案比較友善。
所以在實際上的應用，可以

>- 考慮使用ocr_only抽取文字
>- 然後用hi_res抽取圖像和表格


## PDF -> Vectorstore

你現在有了文字，表格，圖像。你可以開始建立自己專屬的數據庫

### 文字:
- 原始數據
- 進行壓縮 (像是summary)

### 表格/圖像:
- 使用image caption將圖片轉換成文字

將 max_character 和 new_after_n_chars 分別拉高到 2000 和 1500

In [ ]:
elements_strategy_ocr_only = partition_pdf(
    'menu.pdf',
    strategy='ocr_only',
    chunking_strategy='by_title',
    max_characters=2000,
    new_after_n_chars=1500,
)

In [ ]:
import os
import configparser

from langchain_openai import ChatOpenAI


def credential_init():

  credential_file = "credentials.ini"

  if os.path.exists(credential_file):
      credentials = configparser.ConfigParser()
      credentials.read(credential_file)
      os.environ['OPENAI_API_KEY'] = credentials['openai'].get('api_key')
  else:
      os.environ['OPENAI_API_KEY'] = os.environ['OPENAI']

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
           model_name="gpt-4o-mini", temperature=0)

In [ ]:
import io
import base64

from PIL import Image
from langchain_core.prompts.image import ImagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document
from langchain_core.runnables import chain, RunnablePassthrough, RunnableLambda


def build_standard_chat_prompt_template(kwargs):
    messages = []

    if 'system' in kwargs:
        content = kwargs.get('system')

        # allow list of prompts for multimodal
        if isinstance(content, list):
            prompts = [PromptTemplate(**c) for c in content]
        else:
            prompts = [PromptTemplate(**content)]

        message = SystemMessagePromptTemplate(prompt=prompts)
        messages.append(message)

    if 'human' in kwargs:
        content = kwargs.get('human')

        # allow list of prompts for multimodal
        if isinstance(content, list):
            prompts = []
            for c in content:
                if c.get("type") == "image":
                    prompts.append(ImagePromptTemplate(**c))
                else:
                    prompts.append(PromptTemplate(**c))
        else:
            if content.get("type") == "image":
                prompts = [ImagePromptTemplate(**content)]
            else:
                prompts = [PromptTemplate(**content)]

        message = HumanMessagePromptTemplate(prompt=prompts)
        messages.append(message)

    chat_prompt_template = ChatPromptTemplate.from_messages(messages)

    return chat_prompt_template

### 文字壓縮

In [ ]:
prompt = f"Summarize the following text:\n\n{elements_strategy_ocr_only[0]}\n\nSummary:"

In [ ]:
model.invoke(prompt)

In [ ]:
async def summarize_documents(model, elements):
    text_prompt_template = {"template": "Summarize the following text:\n\n{query}\n\nSummary:", "input_variables": ["query"]}
    chat_prompt_template = ChatPromptTemplate.from_messages([
        HumanMessagePromptTemplate(prompt=[PromptTemplate(**text_prompt_template)])
    ])

    text_pipeline = chat_prompt_template | model | StrOutputParser()
    batches = [{"query": c.text} for c in elements]

    return await text_pipeline.abatch(batches)

# 執行
async def main():
    summaries = await summarize_documents(model, elements_strategy_ocr_only)

    return summaries

results = asyncio.run(main())

In [ ]:
documents = []

for result in results:

  documents.append(Document(page_content=result, metadata={'filename': "menu.pdf",
                               'type': "text"}))

In [ ]:
documents[-1]

願意的話，你甚至可以辦到標註是在哪一頁

### 表格和影像提取

你可以注意到有些圖片是False Positive。在務實上可以考慮用Pydantic Model的輸出格式幫忙標記假的圖片。

跟模型說: 你的眼睛業障重，假的。

In [ ]:
import aiofiles
import asyncio
from operator import itemgetter
from pathlib import Path


# Async function to read image and convert to base64

async def image_to_base64(image_path: str) -> str:
    loop = asyncio.get_event_loop()

    def read_image():
        with Image.open(image_path) as image:
            buffered = io.BytesIO()
            image.convert("RGB").save(buffered, format="JPEG")
            return base64.b64encode(buffered.getvalue()).decode('utf-8')

    return await loop.run_in_executor(None, read_image)


text_prompt_template = {"template": "Describe the content."}
image_prompt_template = {"type": "image",
              "template": {"url": "data:image/jpeg;base64,{image_str}"},
              "input_variables": ["image_str"]}

input_ = {
    "human": [text_prompt_template, image_prompt_template],
}

chat_prompt_template = build_standard_chat_prompt_template(input_)

In [ ]:
# --- 使用 RunnableLambda 處理 async 函式 ---
async def make_image_caption_pipeline(model):
    async def assign_image_str(x):
        image_str = await image_to_base64(x['image_path'])
        return {**x, 'image_str': image_str}

    image_2_image_str_chain = RunnableLambda(assign_image_str)
    pipeline = image_2_image_str_chain | chat_prompt_template | model | StrOutputParser()
    return pipeline

In [ ]:
# --- 主流程 ---
async def main():
    model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                       model_name="gpt-4o-mini", temperature=0)
    
    pipeline = await make_image_caption_pipeline(model)
    image_dir = "menu"

    batches = [{"image_path": os.path.join(image_dir, c)}
               for c in os.listdir(image_dir) if 'figure' in Path(c).stem]

    results = await pipeline.abatch(batches)

    return results

# 執行主函式
results = asyncio.run(main())

In [ ]:
for result in results:

    documents.append(Document(page_content=result, metadata={'filename': "menu.pdf",
                               "type": "figure"}))

In [ ]:
documents[-1]

基於Documents內容建立vectorstore

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

### Save the vectorstore

In [ ]:
vectorstore.save_local("vectorstore")

### Load the vectorstore

In [ ]:
vectorstore_1 = FAISS.load_local(
    "vectorstore", embeddings, allow_dangerous_deserialization=True
)

vectorstore_1.docstore._dict

### 合併vectorstore


In [ ]:
vectorstore.merge_from(vectorstore_1)

---

In [ ]:
os.makedirs("codex")

In [ ]:
async def pdf_2_vectorstore(filename, embeddings):

  image_dir = Path(filename).stem

  elements = partition_pdf(
    filename,
    chunking_strategy='by_title',
    extract_image_block_types=["Table"],
    # infer_table_structure=False,
    form_extraction_skip_tables=True,
    max_characters=4000,
    new_after_n_chars=3800,
    strategy='hi_res',
    extract_image_block_output_dir=image_dir
)

  documents = []

  text_prompt_template = {"template": "Summarize the following text:\n\n{query}\n\nSummary:", "input_variables": ["query"]}

  input_ = {
      "human": [text_prompt_template],
  }

  chat_prompt_template = build_standard_chat_prompt_template(input_)

  text_pipeline = chat_prompt_template|model|StrOutputParser()

  batches = [{"query": c.text} for c in elements]

  results = await text_pipeline.abatch(batches)

  for result in results:

    documents.append(Document(page_content=result, metadata={'filename': image_dir,
                                 'type': "text"}))


  text_prompt_template = {"template": "Describe the content."}
  image_prompt_template = {"type": "image",
                "template": {"url": "data:image/jpeg;base64,{image_str}"},
                "input_variables": ["image_str"]}

  input_ = {
      "human": [text_prompt_template, image_prompt_template],
  }

  chat_prompt_template = build_standard_chat_prompt_template(input_)

  # Async chain for converting image_path -> base64
  # The lambda in RunnablePassthrough.assign is async-friendly\

  image_2_image_str_chain = RunnablePassthrough.assign(image_str=lambda x: itemgetter('image_path')|image_to_base64)


  image_caption_pipeline = image_2_image_str_chain | chat_prompt_template| model | StrOutputParser()

  batches = [{"image_path": os.path.join(image_dir, c)} for c in os.listdir(image_dir) if 'table' in Path(c).stem]

  results = await image_caption_pipeline.abatch(batches)

  for result in results:

documents.append(Document(page_content=result, metadata={'filename': image_dir,
                                "type": "table"}))

  vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

  return vectorstore

In [ ]:
from operator import itemgetter
from pathlib import Path
from tqdm import tqdm

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore_list = []

for filename in tqdm(os.listdir("codex")):

  print(f"\n****\n{filename}\n****\n")

  vectorstore = await pdf_2_vectorstore(os.path.join("codex", filename), embeddings)

  vectorstore_list.append(vectorstore)

In [ ]:
merged_store = vectorstore_list[0]
for vs in vectorstore_list[1:]:
  merged_store.merge_from(vs)

In [ ]:
merged_store.save_local("warhammer40k_codex")